### architecture thoughts

- 100% container based deployment
    - multiple containers per pi, distinguished by port
        - this could be a way of restoring previous behavior at a pi
    - container synonymous with mothership
- some machines (not necessarily pi's) could host multiple motherships 
    - these could be repositories for libraries of motherships, to be used as templates for pi's in the field
- a fleet is a collection of motherships
    - perhaps the fleet api would listen on 5050
        - the others listening on some other port
    - maybe an associated restful api that manages the fleet (a collection of mothership hosts)
    - there is a mothership sdk. there will be a fleet sdk.
    - actions would include
        - use a mothership's configuration to replace the configurations in other motherships
    - there would be a mothership discovery function
        - this would allow any machine to take of the role of fleet manager (taking fleet api requests)
        - plus a mothership health monitoring function
